### Setup and Imports

In [2]:
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, LongType, DoubleType

# Spark MLlib Imports
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer, IndexToString
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, ClusteringEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Initialize Spark Session with "Intermediate" settings
spark = SparkSession.builder.getOrCreate()

print(f"Spark Version: {spark.version}")

# Define paths
DATA_DIR = "data_cleaned"
MODELS_DIR = "models"

if os.path.exists(MODELS_DIR):
    shutil.rmtree(MODELS_DIR)
os.makedirs(MODELS_DIR)

print("✅ Setup Complete (Intermediate Power Mode).")

Spark Version: 3.5.0
✅ Setup Complete (Intermediate Power Mode).


### Robust Data Loading

In [4]:
# Load all posts data
posts_path = f"{DATA_DIR}/*_posts.csv"

# 1. Read as strings
df_raw = spark.read.csv(posts_path, header=True, inferSchema=False)

# 2. Define Valid Subreddits (The Allow-List)
valid_subreddits = [
    'Conservative', 'Libertarian', 'PoliticalDiscussion', 
    'neutralnews', 'politics', 'socialism', 'worldnews'
]

# 3. Robust cleaning and FILTERING
df_clean = df_raw.withColumn(
    "timestamp", F.to_timestamp(F.from_unixtime(F.expr("try_cast(created_utc AS LONG)")))
).withColumn(
    "score", F.expr("try_cast(score AS INT)")
).withColumn(
    "num_comments", F.expr("try_cast(num_comments AS INT)")
).filter(
    F.col("subreddit").isin(valid_subreddits)  # <--- CRITICAL FIX: Drop rows with garbage subreddits
).dropna(subset=["title", "subreddit"])

# 4. Select only what we need for ML
df_ml = df_clean.select("title", "subreddit")

print(f"Total records for ML (Cleaned): {df_ml.count():,}")
df_ml.show(5, truncate=False)

Total records for ML (Cleaned): 32,233
+-----------------------------------------------------------------------------------------------------+---------+
|title                                                                                                |subreddit|
+-----------------------------------------------------------------------------------------------------+---------+
|Parishioners hold prayer at Naval Station Great Lakes ahead of planned immigration crackdown         |politics |
|What challenges are their in your mind leaning on capitalism, socialism, and communism?              |politics |
|RFK Jr, who doesn’t have a medical degree, says he can diagnose children just by just looking at them|politics |
|'Something is very wrong': Ex-Trump insider flags 'alarming' news about president's health           |politics |
|Florida Democrat: DeSantis, Trump ‘declared war on people of color’                                  |politics |
+------------------------------------------------

### Handling Class Imbalance

In [5]:
# Spark MLlib doesn't handle class imbalance automatically like sklearn's class_weight='balanced'.
# We must calculate weights manually and add them as a column.

# 1. Count totals per class
class_counts = df_ml.groupBy("subreddit").count().collect()
total_count = df_ml.count()
num_classes = len(class_counts)

# 2. Calculate weights: Weight = Total / (NumClasses * ClassCount)
# This gives higher weight to minority classes
balancing_ratios = {}
for row in class_counts:
    balancing_ratios[row['subreddit']] = total_count / (num_classes * row['count'])

print("Class Weights:")
for sub, weight in balancing_ratios.items():
    print(f"  {sub}: {weight:.4f}")

# 3. Create a function to map subreddit to weight
def get_weight(subreddit):
    return float(balancing_ratios.get(subreddit, 1.0))

# Register UDF
weight_udf = F.udf(get_weight, DoubleType())

# 4. Add weight column
from pyspark.sql.types import DoubleType
df_weighted = df_ml.withColumn("classWeight", weight_udf(F.col("subreddit")))

# 5. Split into Train/Test
train_data, test_data = df_weighted.randomSplit([0.8, 0.2], seed=42)
print(f"Training Set: {train_data.count():,} | Test Set: {test_data.count():,}")

Class Weights:
  politics: 0.3587
  Conservative: 0.4147
  worldnews: 0.8576
  socialism: 3.9593
  PoliticalDiscussion: 4.5456
  Libertarian: 7.7914
  neutralnews: 29.9007
Training Set: 25,853 | Test Set: 6,380


### Building the Feature Engineering Pipeline

In [15]:
# We define the stages that transform text into numbers.
# These stages will be shared across our supervised models.

# 1. Convert target string (subreddit) to index (0, 1, 2...)
label_indexer = StringIndexer(inputCol="subreddit", outputCol="label").fit(train_data)

# 2. Tokenize: Split sentences into words
tokenizer = Tokenizer(inputCol="title", outputCol="words")

# 3. Remove Stop Words: Remove "the", "a", "is", etc.
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# 4. TF (HashingTF): Convert words to feature vectors (raw counts)
# numFeatures=10000 matches our Phase 2 logic
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=10000)

# 5. IDF: Rescale features based on how rare they are across documents
idf = IDF(inputCol="rawFeatures", outputCol="features")

# 6. Convert prediction index back to string (for human readable output)
# We need the labels from the indexer
label_converter = IndexToString(inputCol="prediction", outputCol="predicted_subreddit", labels=label_indexer.labels)

print("✅ Feature Engineering stages defined.")

✅ Feature Engineering stages defined.


### Additional Memory Optimization

In [5]:
# # Reduce Spark memory overhead
# spark.conf.set("spark.sql.shuffle.partitions", "8")  # Default is 200
# spark.conf.set("spark.default.parallelism", "4")

# # Unpersist cached data if you cached train_data earlier
# train_data.unpersist()


DataFrame[title: string, subreddit: string, classWeight: double]

### Model 1 - Logistic Regression (with Cross-Validation)

In [10]:
print("--- Training Model 1: Logistic Regression ---")

# 1. Define the Classifier
# Note: We pass the 'classWeight' column we created earlier
lr = LogisticRegression(featuresCol="features", labelCol="label", weightCol="classWeight")

# 2. Build the Pipeline
lr_pipeline = Pipeline(stages=[label_indexer, tokenizer, remover, hashingTF, idf, lr, label_converter])

# 3. Define Hyperparameter Grid for Cross-Validation
# We will tune the regularization parameter (regParam)
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5]) \
    .build()

# 4. Set up CrossValidator (3-fold)
crossval = CrossValidator(estimator=lr_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy"),
                          numFolds=3)

# 5. Train (Fit)
print("Training with Cross-Validation (this may take a minute)...")
cv_model_lr = crossval.fit(train_data)
best_lr_model = cv_model_lr.bestModel

# 6. Evaluate
predictions_lr = best_lr_model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy_lr = evaluator.evaluate(predictions_lr)
print(f"✅ Logistic Regression Accuracy: {accuracy_lr:.4f}")

# 7. Save the Model (Crucial for MCP)
model_path = f"{MODELS_DIR}/spark_lr_model"
best_lr_model.save(model_path)
print(f"💾 Model saved to: {model_path}")

--- Training Model 1: Logistic Regression ---
Training with Cross-Validation (this may take a minute)...
✅ Logistic Regression Accuracy: 0.5671
💾 Model saved to: models/spark_lr_model


### Model 2 - Naive Bayes

In [11]:
print("--- Training Model 2: Naive Bayes ---")

# 1. Define Classifier
# Naive Bayes in Spark does not support weightCol, but is naturally robust.
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol="features", labelCol="label")

# 2. Build Pipeline
nb_pipeline = Pipeline(stages=[label_indexer, tokenizer, remover, hashingTF, idf, nb, label_converter])

# 3. Train
nb_model = nb_pipeline.fit(train_data)

# 4. Evaluate
predictions_nb = nb_model.transform(test_data)
accuracy_nb = evaluator.evaluate(predictions_nb)
print(f"✅ Naive Bayes Accuracy: {accuracy_nb:.4f}")

# 5. Save Model
model_path = f"{MODELS_DIR}/spark_nb_model"
nb_model.save(model_path)
print(f"💾 Model saved to: {model_path}")

--- Training Model 2: Naive Bayes ---
✅ Naive Bayes Accuracy: 0.5464
💾 Model saved to: models/spark_nb_model


### Model 3 - K-Means Clustering (Unsupervised)

In [19]:
print("--- Training Model 3: K-Means Clustering (In-Memory) ---")

# 1. Split Data
train_set, val_set, test_set = df_weighted.randomSplit([0.7, 0.15, 0.15], seed=42)

# --- CRITICAL STEP: Cache and Force Materialization ---
# We cache the training set to RAM.
# We MUST call .count() immediately to force Spark to compute the split 
# and store it in memory right now. This prevents the "zip RDD" error.
train_set.cache()
print(f"Materializing training data into RAM... Count: {train_set.count():,}")

# 2. Define K-Means
# k=7 (for 7 subreddits)
kmeans = KMeans(featuresCol="features", k=7, seed=42)

# 3. Build Pipeline
# Note: We don't need label_indexer or label_converter for unsupervised learning
kmeans_pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, kmeans])

# 4. Train
print("Training K-Means...")
kmeans_model = kmeans_pipeline.fit(train_set)
print("✅ Training Complete.")

# 5. Evaluate (Silhouette Score)
predictions_km = kmeans_model.transform(test_set)
evaluator_km = ClusteringEvaluator()
silhouette = evaluator_km.evaluate(predictions_km)
print(f"✅ K-Means Silhouette Score: {silhouette:.4f}")

# 6. Save Model
model_path = f"{MODELS_DIR}/spark_kmeans_model"
kmeans_model.save(model_path)
print(f"💾 Model saved to: {model_path}")

# 7. Clean up RAM
train_set.unpersist()
print("🧹 RAM cleared.")

--- Training Model 3: K-Means Clustering (In-Memory) ---
Materializing training data into RAM... Count: 22,696
Training K-Means...
✅ Training Complete.
✅ K-Means Silhouette Score: 0.0141
💾 Model saved to: models/spark_kmeans_model
🧹 RAM cleared.


### Verify Saved Models (Dry Run Check)

In [20]:
print("--- Verifying Model Loading ---")

from pyspark.ml import PipelineModel
from pyspark.sql.functions import lit

try:
    # 1. Load the model
    loaded_lr = PipelineModel.load(f"{MODELS_DIR}/spark_lr_model")
    print("✅ Successfully loaded Logistic Regression model.")
    
    # 2. Create sample data
    sample_data = [
        ("Finland turns down US request for eggs",),
        ("I don't want $5,000 from DOGE ",),
        ("Today's Russia is the anti thesis of everything USSR once was.",)
    ]
    sample_df = spark.createDataFrame(sample_data, ["title"])
    
    # Add a dummy 'subreddit' column ---
    # The pipeline expects this column to exist to pass the StringIndexer stage.
    # We fill it with a dummy value (e.g., "politics" or null).
    sample_df_ready = sample_df.withColumn("subreddit", lit("politics"))
    
    # 3. Run Prediction
    result = loaded_lr.transform(sample_df_ready)
    
    # 4. Extract result
    # prediction = result.select("predicted_subreddit").first()[0]
    predictions = result.select("title", "predicted_subreddit").collect()
    # print(f"   Test Prediction for '{sample_data[0][0]}': r/{prediction}")
    print("\n📊 Predictions:")
    for i, row in enumerate(predictions, 1):
        print(f"   {i}. \"{row['title']}\"")
        print(f"      → Predicted: r/{row['predicted_subreddit']}\n")
    
except Exception as e:
    print(f"❌ Failed to load or run model: {e}")

--- Verifying Model Loading ---
✅ Successfully loaded Logistic Regression model.

📊 Predictions:
   1. "Finland turns down US request for eggs"
      → Predicted: r/worldnews

   2. "I don't want $5,000 from DOGE "
      → Predicted: r/socialism

   3. "Today's Russia is the anti thesis of everything USSR once was."
      → Predicted: r/Conservative

